In [ ]:
# Notebook Parameters
output_dir = ""
requirements_txt = ""

In [ ]:
# Google Colab Setup
if "google.colab" in str(get_ipython()):
    # mount google drive path
    from google.colab import drive 
    drive.mount('/content/drive', force_remount=False)
    # install project requirements.txt in colab env
    if requirements_txt:
        !pip install -r {requirements_txt}

In [ ]:
import os, glob
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

## setup path and load data

In [ ]:
# if output_dir is not defined, fallback to climate machine settings or cwd
if not output_dir:
    output_dir = os.environ.get("CLIMATEMACHINE_SETTINGS_OUTPUT_DIR", os.getcwd())

# load aux diagnostics state
nc_aux_datasets = glob.glob(os.path.join(output_dir, "**", "RisingBubble_DumpAux*.nc"), recursive=True)
if not nc_aux_datasets:
    raise Exception("No NetCDF DumpAux dataset found under path prefix: `{}`".format(output_dir))
print(nc_aux_datasets)

# load diagnostics state
nc_state_datasets = glob.glob(os.path.join(output_dir, "**", "RisingBubble_DumpState*.nc"), recursive=True)
if not nc_state_datasets:
    raise Exception("No NetCDF DumpState dataset found under path prefix: `{}`".format(output_dir))
print(nc_state_datasets)

In [ ]:
# load data
auxdata = nc.Dataset(nc_aux_datasets[0], "r");
statedata = nc.Dataset(nc_state_datasets[0], "r");

In [ ]:
t = statedata["time"][:];
x = statedata["x"][:];
y = statedata["y"][:];
z = statedata["z"][:];

xlen = len(x)
ylen = len(y)
zlen = len(z)

ρ = statedata["ρ"][:].mean(axis=2)
ρ_ref = auxdata["ref_state.ρ"][:].mean(axis=2)

θ_v = auxdata["moisture.θ_v"][:].mean(axis=2)

ρ_a = ρ[:] - ρ_ref[:]
b = -ρ_a/ρ[:]

w = (statedata['ρu[3]'][:].mean(axis=2))/ρ[:]

## check initial theta and density profile

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,5))

C=ax[0].contourf(x,z, θ_v[0,:,:], cmap='RdBu_r')
plt.colorbar(C, ax=ax[0])
ax[0].set_title('initial theta')

tm = abs(ρ_a[0,:,:]).max()
C=ax[1].contourf(x,z, ρ_a[0,:,:], vmin=-tm, vmax=tm, cmap='RdBu_r')
plt.colorbar(C, ax=ax[1])
ax[1].set_title('initial density')

plt.savefig('./risingbubble-initial.png', facecolor='w', transparent=False)

## check snapshots of every 200 ssecs

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(18,15))

for ii, it in enumerate(np.arange(1, 7, 2)):
    
    tm = abs(ρ_a[it,:,:]).max()
    C=ax[0,ii].contourf(x,z, ρ_a[it,:,:], vmin=-tm, vmax=tm, cmap='RdBu_r')
    plt.colorbar(C, ax=ax[0,ii])
    ax[0,ii].set_title('density at t='+str(int(t[it]))+' sec')
    
    tm = abs(b[it,:,:]).max()
    C=ax[1,ii].contourf(x,z, b[it,:,:], vmin=-tm, vmax=tm, cmap='RdBu_r')
    plt.colorbar(C, ax=ax[1,ii])
    ax[1,ii].set_title('buoyancy at t='+str(int(t[it]))+' sec')
    
    tm = abs(w[it,:,:]).max()
    C=ax[2,ii].contourf(x,z, w[it,:,:], vmin=-tm, vmax=tm, cmap='RdBu_r')
    plt.colorbar(C, ax=ax[2,ii])
    ax[2,ii].set_title('w at t='+str(int(t[it]))+' sec')

plt.savefig('./risingbubble-snapshots.png', facecolor='w', transparent=False)